In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_4월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 18281/18281 [37:13<00:00,  8.18it/s]  


,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/wltjsdl42/222716869226,Monthly Diary : April 2022.,2022.04.30.,04.01\n생일 맞이 강원도 여행의 시작!\n후후 제일 기대하던 여행!!\n하지만...,0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,[(04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 ...
1,https://blog.naver.com/starrysson/222716868397,파주 나들이 헤이리마을 브런치는 카페귀천,2022.04.30.,드라이브 코스\n[카페 귀천]\n천상병 시인을 알아보는 곳\n위치/영업일/휴무일/메...,1,드라이브 코스 카페 귀천 천상병 시인을 알아보는 곳 위치 영업일 휴무일 메뉴 지난 ...,[(드라이브 코스 카페 귀천 천상병 시인을 알아보는 곳 위치 영업일 휴무일 메뉴 지...
2,https://blog.naver.com/embassy_tn/222716868029,"[튀니지 여행] 카이로우안 여행(대모스크, 시디 사하브 자위야...",2022.04.30.,안녕하세요. 항상 화eating하는 카림 1기의 오지선입니다:)\n‘튀니지 여행’은...,2,안녕하세요 항상 화eating하는 카림 1기의 오지선입니다 튀니지 여행 은 튀니지의...,"[(안녕하세요 항상 화eating하는 카림 1기의 오지선입니다, 0, 32, Non..."
3,https://blog.naver.com/tndusl2501/222716867616,"순천 여행 :: 순천 선암사, 겹벚꽃 구경 실패?!, 순천 가볼 만한...",2022.04.30.,안녕하세요.. 도도한 쑤언니입니다 :D\n작년에 순천 선암사로 겹벚꽃을 구경하러 갔...,3,안녕하세요 도도한 쑤언니입니다 D 작년에 순천 선암사로 겹벚꽃을 구경하러 갔는데 꽃...,"[(안녕하세요 도도한 쑤언니입니다, 0, 16, None, []), (D 작년에 순..."
4,https://blog.naver.com/jyeon_83/222716864076,겨울 지나 벚꽃,2022.04.30.,"새벽에 쓰다가 한번 날렸습니다 네네..;;\n4월도 끝났네여~,,\n참 한달 한달 ...",4,새벽에 쓰다가 한번 날렸습니다 네네 4월도 끝났네여 참 한달 한달 느린거같으면서도 ...,"[(새벽에 쓰다가 한번 날렸습니다, 0, 16, None, []), (네네 4월도 ..."
...,...,...,...,...,...,...,...
18276,https://blog.naver.com/dsjkfjksfjo/222688429918,스타일 라디오 최저가 공구 무료배송,2022.04.01.,블로그\n게시판 72개의 글\n게시판\n목록열기\n게시판\n스타일 라디오 최저가 공...,18276,블로그 게시판 72개의 글 게시판 목록열기 게시판 스타일 라디오 최저가 공구 무료배...,[(블로그 게시판 72개의 글 게시판 목록열기 게시판 스타일 라디오 최저가 공구 무...
18277,https://blog.naver.com/mcho1234/222683862874,엽기獵奇(사냥/전쟁 + 기적 = 천지창조) Sleeping Beauty...,2022.04.01.,영화 엽기獵奇적인 그녀에서 어떤 인물을 말할때 엽기적이다고 하는 가? 한자 엽기獵奇...,18277,영화 엽기 적인 그녀에서 어떤 인물을 말할때 엽기적이다고 하는 가 한자 엽기 의 의...,[(영화 엽기 적인 그녀에서 어떤 인물을 말할때 엽기적이다고 하는 가 한자 엽기 의...
18278,https://blog.naver.com/zodzk_2001/222688427612,울산데이트 울산 벚꽃 동백꽃 명소 MBC문화동산,2022.04.01.,2022년은 더 좋은 사람들과\n더 많은 꽃구경을 다닐 수 있도록🌸💕\n#울산벚꽃명...,18278,2022년은 더 좋은 사람들과 더 많은 꽃구경을 다닐 수 있도록 울산벚꽃명소 유명한...,[(2022년은 더 좋은 사람들과 더 많은 꽃구경을 다닐 수 있도록 울산벚꽃명소 유...
18279,https://blog.naver.com/tjdnswm010/222688425217,2022.3월전시추천:구찌전시회_구찌가든 아키타이프_ 절대적...,2022.04.01.,3월이 다 지나가고\n전시도 막을 내렸지만\n블로그에 포스팅하는 이유는 전시가 너무...,18279,3월이 다 지나가고 전시도 막을 내렸지만 블로그에 포스팅하는 이유는 전시가 너무 좋...,[(3월이 다 지나가고 전시도 막을 내렸지만 블로그에 포스팅하는 이유는 전시가 너무...


In [5]:
import pickle
# 데이터프레임을 피클 파일로 저장함으로써 데이터타입 보존
with open('4월_문장분리.pkl', 'wb') as file:
    pickle.dump(df, file)

In [2]:
import pickle
# 저장된 피클 파일을 불러오기 익일 재실행 경우, 이 코드만 돌리면 시간절약 가능.
with open('4월_문장분리.pkl', 'rb') as file:
    df = pickle.load(file)
df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/wltjsdl42/222716869226,Monthly Diary : April 2022.,2022.04.30.,04.01\n생일 맞이 강원도 여행의 시작!\n후후 제일 기대하던 여행!!\n하지만...,0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,[(04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 ...
1,https://blog.naver.com/starrysson/222716868397,파주 나들이 헤이리마을 브런치는 카페귀천,2022.04.30.,드라이브 코스\n[카페 귀천]\n천상병 시인을 알아보는 곳\n위치/영업일/휴무일/메...,1,드라이브 코스 카페 귀천 천상병 시인을 알아보는 곳 위치 영업일 휴무일 메뉴 지난 ...,[(드라이브 코스 카페 귀천 천상병 시인을 알아보는 곳 위치 영업일 휴무일 메뉴 지...
2,https://blog.naver.com/embassy_tn/222716868029,"[튀니지 여행] 카이로우안 여행(대모스크, 시디 사하브 자위야...",2022.04.30.,안녕하세요. 항상 화eating하는 카림 1기의 오지선입니다:)\n‘튀니지 여행’은...,2,안녕하세요 항상 화eating하는 카림 1기의 오지선입니다 튀니지 여행 은 튀니지의...,"[(안녕하세요 항상 화eating하는 카림 1기의 오지선입니다, 0, 32, Non..."
3,https://blog.naver.com/tndusl2501/222716867616,"순천 여행 :: 순천 선암사, 겹벚꽃 구경 실패?!, 순천 가볼 만한...",2022.04.30.,안녕하세요.. 도도한 쑤언니입니다 :D\n작년에 순천 선암사로 겹벚꽃을 구경하러 갔...,3,안녕하세요 도도한 쑤언니입니다 D 작년에 순천 선암사로 겹벚꽃을 구경하러 갔는데 꽃...,"[(안녕하세요 도도한 쑤언니입니다, 0, 16, None, []), (D 작년에 순..."
4,https://blog.naver.com/jyeon_83/222716864076,겨울 지나 벚꽃,2022.04.30.,"새벽에 쓰다가 한번 날렸습니다 네네..;;\n4월도 끝났네여~,,\n참 한달 한달 ...",4,새벽에 쓰다가 한번 날렸습니다 네네 4월도 끝났네여 참 한달 한달 느린거같으면서도 ...,"[(새벽에 쓰다가 한번 날렸습니다, 0, 16, None, []), (네네 4월도 ..."
...,...,...,...,...,...,...,...
18276,https://blog.naver.com/dsjkfjksfjo/222688429918,스타일 라디오 최저가 공구 무료배송,2022.04.01.,블로그\n게시판 72개의 글\n게시판\n목록열기\n게시판\n스타일 라디오 최저가 공...,18276,블로그 게시판 72개의 글 게시판 목록열기 게시판 스타일 라디오 최저가 공구 무료배...,[(블로그 게시판 72개의 글 게시판 목록열기 게시판 스타일 라디오 최저가 공구 무...
18277,https://blog.naver.com/mcho1234/222683862874,엽기獵奇(사냥/전쟁 + 기적 = 천지창조) Sleeping Beauty...,2022.04.01.,영화 엽기獵奇적인 그녀에서 어떤 인물을 말할때 엽기적이다고 하는 가? 한자 엽기獵奇...,18277,영화 엽기 적인 그녀에서 어떤 인물을 말할때 엽기적이다고 하는 가 한자 엽기 의 의...,[(영화 엽기 적인 그녀에서 어떤 인물을 말할때 엽기적이다고 하는 가 한자 엽기 의...
18278,https://blog.naver.com/zodzk_2001/222688427612,울산데이트 울산 벚꽃 동백꽃 명소 MBC문화동산,2022.04.01.,2022년은 더 좋은 사람들과\n더 많은 꽃구경을 다닐 수 있도록🌸💕\n#울산벚꽃명...,18278,2022년은 더 좋은 사람들과 더 많은 꽃구경을 다닐 수 있도록 울산벚꽃명소 유명한...,[(2022년은 더 좋은 사람들과 더 많은 꽃구경을 다닐 수 있도록 울산벚꽃명소 유...
18279,https://blog.naver.com/tjdnswm010/222688425217,2022.3월전시추천:구찌전시회_구찌가든 아키타이프_ 절대적...,2022.04.01.,3월이 다 지나가고\n전시도 막을 내렸지만\n블로그에 포스팅하는 이유는 전시가 너무...,18279,3월이 다 지나가고 전시도 막을 내렸지만 블로그에 포스팅하는 이유는 전시가 너무 좋...,[(3월이 다 지나가고 전시도 막을 내렸지만 블로그에 포스팅하는 이유는 전시가 너무...


In [3]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [8]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_4월_전처리.csv", index=False, encoding='UTF-8')

In [4]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_4월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0
...,...,...
510935,이번 꽃 심기 행사는 민주적인 학교문화 만들기의 과정으로 학교 정문 화단에 우정 이...,11188
510936,진영제일고 취업 캠프로 취업의 날개를 달다 2022 04 08 진영제일고등학교 복도...,11188
510937,취업 캠프는 학생들의 취업역량 강화를 위해 학년별 특성에 맞게 서로 다른 프로그램을...,11188
510938,다음 주에도 더욱 풍성한 경남 학교 소식들로 만나뵐게요,11188


In [5]:
df2 = df2[df2['org_idx'] < 11150]
df2

,split_str,org_idx
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0
...,...,...
509224,그녀는 우리가 왜 하면 안 되는 줄 알면서도 과식하는지 폭넓고 깊이 있게 설명한다,11149
509225,무엇보다 대안을 제시한다 얼토당토않거나 실현 가능성 없는 대안이 아니라 충분히 납득...,11149
509226,미루지 마라 음식과의 관계는 인생에서 가장 중요한 관계일 수 있다,11149
509227,음식과의 관계가 삐걱거리면 사랑이 넘치는 배우자나 부모가 되기 어렵고 주어진 일을 ...,11149


In [6]:
for a in range(224,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_4월_전처리.csv", index=False, encoding='UTF-8')
df2

100%|██████████| 31/31 [01:48<00:00,  3.50s/it]


,split_str,org_idx
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0
...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280


In [7]:
df2

,split_str,org_idx
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0
...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280


In [12]:
df2.isnull().sum()

split_str    0
org_idx      0
dtype: int64

In [8]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_4월_전처리.csv", index=False, encoding='UTF-8')